1- Open Anaconda Prompt (for windows -> Start Menu / Anaconda3 / Anaconda Prompt )

2- In Anaconda Prompt, type commands to install necessary libraries: 

    " pip install opencv-python " 
     
      OR
         
    " pip install opencv-contrib-python " (it includes all OpenCV functionality and is RECOMMENDED ! )
     
    Note: You DO NOT want to install both opencv-python and opencv-contrib-python — pick ONE of them.

Source : https://pysource.com/2020/04/02/train-yolo-to-detect-a-custom-object-online-with-free-gpu/


In [1]:
import cv2
import numpy as np
import glob
import random

In [2]:
# Darknet ile eğittimiz ağırlıkları "WEIGHTS" klasörüne taşıyoruz ve ağırlığın ismini "weight" değişkenine yazıyoruz

weight="yolov3_training_7000.weights"
cfg="yolov3_testing.cfg"
net = cv2.dnn.readNet("WEIGHTS/"+weight, "CFGS/"+cfg)     # Networku oluşturuyor.

In [3]:
# Sınıf ismini yazıyoruz. Bizim sadece bir sınıfımız vardı(Drone) ve sınıf numarası ise 0 dır.
# Buradaki sınıf ismi tespit edilen nesnenin görüntüde üzerine yazılması içindir. 
# Etiketleme yaparken bizim tek sınıfımız vardı ve numarası 0 idi. Buradaki "classes" listesinin 0.indisine denk gelmektedir.

classes = ["Drone"]

In [4]:
# Test edilecek resimleri "IMAGES" klasörüne taşıyın ve resim formatlarını aşağıya yazın.

image_fromat="jpg"
images_path = glob.glob("IMAGES/*."+image_fromat)


In [5]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()] # Networkun çıkış katmanlarını seçiyor
colors = np.random.uniform(0, 255, size=(len(classes), 3))   # Class sayısı kadar farklı renkler üretiyor

In [7]:
# Test edeceğimiz resimlerim yollarını rastgele sıralıyor. Bu her defasında farklı sıralamayla resimleri test etmesi için
random.shuffle(images_path)
# Resimleri döngüye sokuyor
for img_path in images_path:
    # resimi yükle
    img = cv2.imread(img_path)
    #img = cv2.resize(img, None, fx=0.6, fy=0.6)
    img = cv2.resize(img , (608,608))  # resmi 608x608 boyutlandırır. Çıktı da bu boyutta olacaktır.

    height, width, channels = img.shape

    # Obje takibini çalıştır
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False) 
    net.setInput(blob)
    outs = net.forward(output_layers) 

    # Sonuçları frame yazdırır
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            label=label+" "+str(round(confidences[i], 3))
            color = colors[class_ids[i]] 
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y -5), font, 1, color, 2)


    cv2.imshow("Image", img)
    key = cv2.waitKey(0)
    if key==27:
        break
cv2.destroyAllWindows()

0
0
[[1]]
0
0
[[0]]
0
[[0]]
()
0
0
0
0
[[2]
 [3]
 [0]]
0
[[0]]
0
[[0]]
0
[[0]]
0
0
[[1]]
()
0
0
[[1]]
()
()
()
()
